# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 8. Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию о пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

In [2]:
# выключаем предупреждения о pandas
pd.set_option('chained_assignment', None)

Для начала давайте посмотрим на документацию к API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.

По [инструкции]() мы можем получить доступ к API, создать приложение и забрать сервисный ключ *(service token)*. Если приложение уже создано, его можно найти в перечне, доступном по этой [ссылке](https://dev.vk.com/ru/admin/apps-list).

In [3]:
# вставить сервисный ключ (service token)
serv_token = "a8268bfca8268bfca8268bfc25ab32cd75aa826a8268bfccc521a030b3c9d65b75a268f"

Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [4]:
v = "5.199"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Знакомая нам функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [5]:
params_wall = {"access_token" : serv_token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [6]:
req_wall = requests.get(main_wall, params = params_wall)

In [7]:
json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [8]:
nposts = json_wall['response']['count']
print(nposts)

1974


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [9]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [10]:
i = items_wall[3]
# i

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [11]:
id_ = i["id"]
date = i["date"]
text = i["text"]
nlikes = i["likes"]["count"]
nreposts = i["reposts"]["count"]
ncomments = i["comments"]["count"]

print(id_, date, nlikes, nreposts, ncomments)
print(text)

4798 1740739205 21 2 0
🚀 Встречаем весну с фестивалем BOULDERHOUSE ФЕСТ 9.0!

С 3 по 16 марта 2025 в Rock Zona — 13 дней лазания, 50 новых трасс от 5A до 7C!

Трассы будут добавляться в три этапа: 

1. В понедельник 03.03.2025 в 15:00 
2. Во вторник 04.03.2025 в 15:00 
3. Во вторник 11.03.20-25 в 15:00

📍 Как принять участие?

🔸 Просто приходите на скалодром, регистрируйтесь у администратора, берите карточку и лезьте!
🔸 Участие = обычной стоимости посещения или групповой тренировки.

🏆 Как считаются результаты?

🔸 Все результаты вносятся онлайн http://rockzona.climbingcompetition.ru/
🔸 Победителем считается тот, кто набрал наибольшее кол-во баллов по сумме всех пройденных трасс;
🔸 Стоимость трассы в баллах зависит от количества участников, которые её пролезли. Чем меньше человек пролезли — тем больше баллов. Flash учитывается, попытки - нет. 

🎯 Группы участников:

* CLIMB – для любителей (до 6A+ у женщин / до 6B+ у мужчин).
* CLIMB+ – для тех, кто лезет от 6B (женщины) и от 6C (мужчин

### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [12]:
iterate = nposts // 100 + 1

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [13]:
params_wall_long = {"access_token" : serv_token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [14]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # обновляем список items_all
    # обновляем парметр offset - увеличиваем на 100
    items_all.extend(items_wall_long)
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    
    time.sleep(1.2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Проверяем длину списка – все ли посты собраны:

In [15]:
len(items_all)

1974

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [16]:
df = pd.DataFrame(items_all)
df.head(10)

,inner_type,donut,comments,marked_as_ads,hash,type,push_subscription,attachments,date,edited,...,post_source,post_type,reposts,text,views,carousel_offset,copy_history,geo,signer_id,zoom_text
0,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 1, 'gr...",0,v6uncvBZJ8sLEX5eug,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1741269764,1.741270e+09,...,{'type': 'api'},post,"{'count': 3, 'user_reposted': 0}",🌙 Мунборд — мощный инструмент для развития сил...,{'count': 727},NaN,NaN,NaN,NaN,NaN
1,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,S_z-W_nMeK56zTtehg,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1741098497,NaN,...,{'type': 'api'},post,"{'count': 1, 'user_reposted': 0}","🌿🌸 Весна, новые трассы и фестивальный вайб! 🌸🌿...",{'count': 395},0.0,NaN,NaN,NaN,NaN
2,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 5, 'gr...",0,qes5pP5jEmW0Fhzbcg,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1740927422,NaN,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 3, 'user_reposted': 0}",Справочник идеальных отговорок на скалодроме 🧗...,{'count': 1106},0.0,NaN,NaN,NaN,NaN
3,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,aiqV5fBEo_YYQnoUcQ,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1740739205,NaN,...,{'type': 'api'},post,"{'count': 2, 'user_reposted': 0}",🚀 Встречаем весну с фестивалем BOULDERHOUSE ФЕ...,{'count': 603},NaN,NaN,NaN,NaN,NaN
4,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 2, 'gr...",0,n1BX7E0FUjcxyswsVg,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1740333687,NaN,...,{'type': 'api'},post,"{'count': 6, 'user_reposted': 0}",1. Разминка – для слабаков 👎\n\nТы же не дедуш...,{'count': 1574},NaN,NaN,NaN,NaN,NaN
5,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 1, 'gr...",0,-H-i2VKJkrjJW_WM_A,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1740152553,NaN,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}",Друзья!\n\nНаш скалодром будет закрыт на подго...,{'count': 951},0.0,NaN,NaN,NaN,NaN
6,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,7MBj6_hPu8UrKRY_XA,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1740052110,NaN,...,{'type': 'api'},post,"{'count': 6, 'user_reposted': 0}","Рассмотрим 5 неочевидных ошибок, которые могут...",{'count': 1470},NaN,NaN,NaN,NaN,NaN
7,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,9LmLcLCprcOpSVnOLQ,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1739990751,NaN,...,"{'platform': 'iphone', 'type': 'api'}",post,"{'count': 0, 'user_reposted': 0}",Вести с полей накрутки - вчера наш скалодром п...,{'count': 1001},0.0,NaN,NaN,NaN,NaN
8,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 2, 'gr...",0,umLiTlcKZ3xz6gZMkw,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1739713340,NaN,...,{'type': 'api'},post,"{'count': 19, 'user_reposted': 0}",☀ 08:00 – Проснулся. Оценил вставание с кроват...,{'count': 2115},NaN,NaN,NaN,NaN,NaN
9,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 1, 'gr...",0,Wcs0u382C85gfhC1rQ,post,{'is_subscribed': False},"[{'type': 'photo', 'photo': {'album_id': -7, '...",1739533877,1.739536e+09,...,{'type': 'api'},post,"{'count': 0, 'user_reposted': 0}",Сегодня чудесный повод порадовать своих любимы...,{'count': 657},0.0,NaN,NaN,NaN,NaN


### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [17]:
small = df[["id", "date", "text", "likes", 
            "reposts", "views", "comments"]]

In [18]:
# пишем lambda-функцию, которая принимает на вход ячейку (x)
# и забирает оттуда значение по ключу count
# для nviews пишем условие с if-else на случай пустых ячеек –
# для старых постов данных о просмотрах нет

small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["nreposts"] = small["reposts"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])
small["nviews"] = small["views"].apply(lambda x: x["count"] if type(x) is dict else x)

In [19]:
small.drop(columns = ["reposts", "comments", "likes"], 
           inplace = True)

**Дополнение.** Преобразуем дату из формата POSIX во что-то более привычное:

In [20]:
# unit = 's': переводим из метки, которая является 
# числом секунд с 1 января 1970

small["date_time"] = pd.to_datetime(small['date'], unit = 's')

In [21]:
# забираем части даты-времени в виде строки (тип string)
# %Y – год в четырехзначном виде, %y – год в двузначном виде
# %m – месяц в числовом виде
# %d – день в числовом виде
# %A – день недели полностью, %a – день недели сокращенно
# %H, %M, %S – часы, минуты, секунды

small["year"] = small["date_time"].dt.strftime('%Y')
small["month"] = small["date_time"].dt.strftime('%m')
small["day"] = small["date_time"].dt.strftime('%d')
small["week_day"] = small["date_time"].dt.strftime('%A')
small["time"] = small["date_time"].dt.strftime('%H:%M:%S')

### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [22]:
with_comm = small[small["ncomments"] > 0] 

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [23]:
with_comm10 = with_comm.head(10)

Заберем id постов и преобразуем их в список:

In [24]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [25]:
ids

[4805, 4799, 4795, 4793, 4788, 4785, 4783, 4778, 4769, 4767]

In [26]:
i = 4799

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : serv_token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [27]:
req = requests.get(main_comm, params = params_comm)
req.json()

{'response': {'count': 5,
  'items': [{'id': 4800,
    'from_id': 260091507,
    'date': 1740995010,
    'text': 'Месяц юмора! 😁',
    'post_id': 4799,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 4802,
       'from_id': -38936316,
       'date': 1741005989,
       'text': '[id260091507|Владимир], тренируемся перед 1 апреля, качаем мышцы, отвечающие за смех',
       'post_id': 4799,
       'owner_id': -38936316,
       'parents_stack': [4800],
       'reply_to_user': 260091507,
       'reply_to_comment': 4800,
       'is_from_post_author': True}],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 4801,
    'from_id': 14750305,
    'date': 1741000697,
    'text': 'Ещё.\n- я просто не люблю такие движения.\n- я сейчас просто только выносливость тренирую',
    'post_id': 4799,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 4803,
     

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [28]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : serv_token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [29]:
df_comm = pd.DataFrame(comments_all)

### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [30]:
# переименовываем столбцы в with_comm10, чтобы 
# столбец с id поста назывался везде одинаково
# inplace = True: сохраняем изменения

with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(2)

,post_id,date,text,views,nlikes,nreposts,ncomments,nviews,date_time,year,month,day,week_day,time
0,4805,1741269764,🌙 Мунборд — мощный инструмент для развития сил...,{'count': 727},21,3,1,727.0,2025-03-06 14:02:44,2025,03,06,Thursday,14:02:44
2,4799,1740927422,Справочник идеальных отговорок на скалодроме 🧗...,{'count': 1106},27,3,5,1106.0,2025-03-02 14:57:02,2025,03,02,Sunday,14:57:02


In [31]:
# объединяем with_comm10 и df_comm по столбцу post_id
# left: к левому датафрейму (with_comm10) подтягиваем данные из правого (df_comm)

final = with_comm10.merge(df_comm, on = "post_id", how = "left")

In [32]:
final

,post_id,date_x,text_x,views,nlikes,nreposts,ncomments,nviews,date_time,year,...,day,week_day,time,id,from_id,date_y,text_y,owner_id,parents_stack,thread
0,4805,1741269764,🌙 Мунборд — мощный инструмент для развития сил...,{'count': 727},21,3,1,727.0,2025-03-06 14:02:44,2025,...,06,Thursday,14:02:44,4806,-98648808,1741348659,🧗‍♂ Мунборд 👍💟,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
1,4799,1740927422,Справочник идеальных отговорок на скалодроме 🧗...,{'count': 1106},27,3,5,1106.0,2025-03-02 14:57:02,2025,...,02,Sunday,14:57:02,4800,260091507,1740995010,Месяц юмора! 😁,-38936316,[],"{'count': 1, 'items': [{'id': 4802, 'from_id':..."
2,4799,1740927422,Справочник идеальных отговорок на скалодроме 🧗...,{'count': 1106},27,3,5,1106.0,2025-03-02 14:57:02,2025,...,02,Sunday,14:57:02,4801,14750305,1741000697,Ещё.\n- я просто не люблю такие движения.\n- я...,-38936316,[],"{'count': 1, 'items': [{'id': 4803, 'from_id':..."
3,4799,1740927422,Справочник идеальных отговорок на скалодроме 🧗...,{'count': 1106},27,3,5,1106.0,2025-03-02 14:57:02,2025,...,02,Sunday,14:57:02,4807,-98648808,1741348771,👍💟🧗‍♂,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
4,4795,1740333687,1. Разминка – для слабаков 👎\n\nТы же не дедуш...,{'count': 1574},44,6,2,1574.0,2025-02-23 18:01:27,2025,...,23,Sunday,18:01:27,4796,6276230,1740337424,😂,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
5,4795,1740333687,1. Разминка – для слабаков 👎\n\nТы же не дедуш...,{'count': 1574},44,6,2,1574.0,2025-02-23 18:01:27,2025,...,23,Sunday,18:01:27,4797,103532552,1740407543,🤣🤣🤣🤣🤣🤣🤣🤣🤣,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
6,4793,1740152553,Друзья!\n\nНаш скалодром будет закрыт на подго...,{'count': 951},5,0,1,951.0,2025-02-21 15:42:33,2025,...,21,Friday,15:42:33,4794,14750305,1740316269,Анечка ♥️,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
7,4788,1739713340,☀ 08:00 – Проснулся. Оценил вставание с кроват...,{'count': 2115},90,19,2,2115.0,2025-02-16 13:42:20,2025,...,16,Sunday,13:42:20,4789,260091507,1739787604,"Веселье и ""движуха"", красивые девушки, белый п...",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
8,4788,1739713340,☀ 08:00 – Проснулся. Оценил вставание с кроват...,{'count': 2115},90,19,2,2115.0,2025-02-16 13:42:20,2025,...,16,Sunday,13:42:20,4790,6276230,1739864741,👍😁,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."
9,4785,1739533877,Сегодня чудесный повод порадовать своих любимы...,{'count': 657},23,0,1,657.0,2025-02-14 11:51:17,2025,...,14,Friday,11:51:17,4787,5464275,1739540710,"А вы знали, что этот жест означает сердечко не...",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's..."


## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [33]:
j = 20473269

Сформируем запрос:

In [34]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : serv_token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 20473269,
   'bdate': '25.3.1994',
   'city': {'id': 1, 'title': 'Moscow'},
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'education_status': "Student (Bachelor's)",
     'education_status_id': 3,
     'graduation': 2016,
     'id': 128,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)'}],
   'first_name': 'Alla',
   'last_name': 'Tambovtseva',
   'can_access_closed': True,
   'is_closed': False}]}

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [38]:
user = final["from_id"][1]
user

260091507

In [39]:
params_user = {"access_token" : serv_token,
               "v" : v,
               "user_id" : user,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 260091507,
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'id': 332,
     'name': 'РГТЭУ (бывш. ЗИСТ, МКИ, МКУ, МГУК)'}],
   'first_name': 'Vladimir',
   'last_name': 'Tsepaev',
   'can_access_closed': True,
   'is_closed': False}]}